In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from commando import ComManDo
from commando.evaluation import test_partial
import numpy as np
from sklearn.metrics import confusion_matrix
import torch

2022-01-18 18:44:47.234886: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-01-18 18:44:47.234917: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [3]:
data1 = np.loadtxt("../data/UnionCom/scGEM/GeneExpression.txt")
data2 = np.loadtxt("../data/UnionCom/scGEM/DNAmethylation.txt")
num_aligned = int(.8 * len(data2))
data2, data2_withheld = data2[:num_aligned], data2[num_aligned:]

# Predict Missing Modality

In [4]:
# Align data
cm = ComManDo(aligned_idx=2*(range(num_aligned),))
cm_data = cm.fit_transform(dataset=[data1, data2])

use random seed: 666
Shape of Raw data
Dataset 0: (177, 34)
Dataset 1: (141, 27)
Device: cpu
---------------------------------
Find correspondence between Dataset 1 and Dataset 2
epoch:[100/2000] err:2.6118 alpha:0.7249
epoch:[200/2000] err:2.3555 alpha:0.7101
epoch:[300/2000] err:1.9467 alpha:0.7620
epoch:[400/2000] err:1.6834 alpha:0.7963
epoch:[500/2000] err:1.5589 alpha:0.8142
epoch:[600/2000] err:1.4994 alpha:0.8240
epoch:[700/2000] err:1.4681 alpha:0.8301
epoch:[800/2000] err:1.4497 alpha:0.8344
epoch:[900/2000] err:1.4376 alpha:0.8378
epoch:[1000/2000] err:1.4291 alpha:0.8406
epoch:[1100/2000] err:1.4225 alpha:0.8430
epoch:[1200/2000] err:1.4172 alpha:0.8451
epoch:[1300/2000] err:1.4126 alpha:0.8469
epoch:[1400/2000] err:1.4086 alpha:0.8485
epoch:[1500/2000] err:1.4051 alpha:0.8500
epoch:[1600/2000] err:1.4020 alpha:0.8513
epoch:[1700/2000] err:1.3996 alpha:0.8525
epoch:[1800/2000] err:1.3976 alpha:0.8535
epoch:[1900/2000] err:1.3960 alpha:0.8545
epoch:[2000/2000] err:1.3948 alp

In [5]:
# Compile prediction and true
predicted = cm.model.decoders[1](torch.tensor(cm_data[0][num_aligned:])).detach().cpu().numpy()
predicted = predicted.flatten() > .5
actual = (data2_withheld - np.min(data2_withheld)) / (np.max(data2_withheld) - np.min(data2_withheld))
actual = actual.flatten() > .5

In [6]:
# Produce confusion matrix
# tn fp
# fn tp
confusion_matrix(actual, predicted, labels=(False, True), normalize='true')

array([[0.85626911, 0.14373089],
       [0.30542636, 0.69457364]])